In [1]:
import pandas as pd
import sys
from pathlib import Path
import os
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# Get the current working directory (where the notebook is)
notebook_dir = Path.cwd()

# Move up to the root directory (parent of 'notebooks')
# project_root = notebook_dir.parent.parent
# project_root = notebook_dir
os.chdir(project_root)


NameError: name 'project_root' is not defined

In [ ]:
from analysis.supervised import *
from analysis.models import * 

## Load data

In [ ]:
wei_llama_3bit = project_root / "datasets/_meta-llamaLlama-3.2-3B-Instruct_prompt_res_activations_3011202404:14:34_rated.pkl"

# path = [wild_jailbreak_path, wei_gemma7bit, wei_llama8bit, wei_llama_3bit, wei_mistral8bit][1]

path = [wei_llama_3bit][0]

df = pd.read_pickle(path)

path = str(path)

In [ ]:
model_name, num_layers, rating_column, hf_ident, input_size = get_model_details(path)

layer= num_layers - 4

# layer = 17
activation_column="prompt_res_activations"
num_samples = len(df)
dataset_name = "wildJailBreak" if "wildJailBreak" in str(path) else "wei_dataset"

df["rating_binary"] = df[rating_column].apply(lambda x: 0 if x == "No" else 1)
df = df[~df["jailbreak_prompt_name"].isin(["auto_payload_splitting", "auto_obfuscation", "dev_mode_with_rant", "gcg", "autodan"])]
df["jailbreak_prompt_name"].value_counts()
#shuffle and balance dataset
df = df.sample(frac=1).reset_index(drop=True)

df = balance_dataset(df, "rating_binary")
df["rating_binary"].value_counts()

In [ ]:
# Save tensors
def save_data_and_labels(df, layers, filepath):
    """
    Prepare data by extracting features, melting the dataframe, and splitting into train/val sets.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing the data
        layers (list): List of layers to extract features from
        exclude_prompts (list, optional): List of prompt names to exclude. Defaults to specified prompts.
    
    Returns:
        tuple: (train_data, train_labels, val_data, val_labels) as tensors
    """
    # Extract features for each layer
    for layer in layers:
        df[f'layer_{layer}'] = df.apply(lambda row: extract_layer(row, layer), axis=1)
    
    # Melt the DataFrame
    melted_df = pd.melt(
        df,
        id_vars=["jailbreak_prompt_name", "rating_binary"],
        value_vars=[f'layer_{layer}' for layer in layers],
        var_name="layer",
        value_name="features"
    )
        
    # Prepare labels and data
    labels = melted_df["rating_binary"].to_list()
    data = melted_df["features"].to_list()
    
    # Convert to tensors
    data_tensor = torch.tensor(np.stack(data), dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    dataset_dict = {
            'data': data_tensor,
            'labels': labels_tensor
        }
    torch.save(dataset_dict, filepath)


    tensor_cols = ['prompt_res_activations'] + [col for col in df.columns if col.startswith('layer_')]
    melted_df_no_tensors = melted_df.drop(tensor_cols, axis=1)
    melted_df_no_tensors.to_csv(filepath.replace(".pt", ".csv"), index=True)

filepath = f"datasets/{path}_tensors.pt"
layers = range(0, num_layers)
save_data_and_labels(df, layers, filepath)

In [ ]:
# Test if your tensor saving worked

# Load the saved tensor data
loaded_data = torch.load(filepath)

# Extract data and labels from the loaded dictionary
data_tensor = loaded_data['data']
labels_tensor = loaded_data['labels']

# Print shapes to verify the loading
print(f"Data tensor shape: {data_tensor.shape}")
print(f"Labels tensor shape: {labels_tensor.shape}")

In [ ]:
# Load the saved CSV data
csv_filepath = filepath.replace(".pt", ".csv")
df_loaded = pd.read_csv(csv_filepath, index_col=0)

# Print info to verify loading
print("\nLoaded CSV info:")
print(df_loaded.info())

